In [1]:
!pip install gradio requests pandas


In [7]:
import os

# Run this ONCE at the start of your Colab session if you want a fresh CSV every time
if os.path.exists(CSV_FILE):
    os.remove(CSV_FILE)


In [4]:
import requests
import pandas as pd

GROQ_API_KEY = "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"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
CSV_FILE = 'call_analysis.csv'

def groq_api_request(prompt, response_type):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "openai/gpt-oss-20b", # Adjust to proper Groq model if needed
        "messages": [
            {"role": "system", "content": "Summarize the customer's query." if response_type=="summary" else "Analyze for positive/neutral/negative"},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500,
        "temperature": 0.1
    }
    response = requests.post(GROQ_API_URL, headers=headers, json=data)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

def analyze_transcript(transcript):
    # Getting summary
    summary_prompt = f"Please read and analyze the COMPLETE customer service call transcript below and provide a comprehensive summary in 2-3 sentences that covers the main issue, resolution, and outcome:\n\n{transcript}"
    summary = groq_api_request(summary_prompt, "summary").strip()
    # Getting sentiment
    sentiment_prompt = f"What's the overall sentiment (Positive/Neutral/Negative) of this call?\n{transcript}\nReply only with one word."
    sentiment = groq_api_request(sentiment_prompt, "sentiment").strip()
    # Save to CSV
    new_row = pd.DataFrame([[transcript, summary, sentiment]],
                           columns=['Transcript', 'Summary', 'Sentiment'])
    try:
        df = pd.read_csv(CSV_FILE)
        df = pd.concat([df, new_row], ignore_index=True)
    except FileNotFoundError:
        df = new_row
    df.to_csv(CSV_FILE, index=False)
    return summary, sentiment


In [8]:
import gradio as gr

iface = gr.Interface(
    fn=analyze_transcript,
    inputs=gr.Textbox(lines=8, label="Paste Customer Call Transcript"),
    outputs=[gr.Textbox(label="Summary"), gr.Textbox(label="Sentiment")],
    title="Mini-Tech Challenge Call Analyzer",
    description="Paste a customer call transcript, click submit, and get summary and sentiment. Results appended to a CSV file."
)
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f0f1c04bb59942bd95.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
import gradio as gr
import pandas as pd

# Button to view/download the CSV
def download_csv():
    try:
        df = pd.read_csv(CSV_FILE)
        return df
    except:
        return "No CSV available yet"

gr.Interface(download_csv, inputs=None, outputs="dataframe", title="View/Download CSV Results").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a5587d0d78e60e2d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
from google.colab import files
files.download('call_analysis.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>